In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from trsm import TRSM, combos_6j

In [3]:
import numpy as np
import awkward as ak

In [4]:
%matplotlib widget
import matplotlib.pyplot as plt
from consistent_plots import hist, hist2d

In [4]:
filename = '../../NMSSM_XYH_YToHH_6b_MX_700_MY_400_6jet_testing_set.root'
trsm_set = TRSM(filename=filename)

In [5]:
tag = '20210816_5btag_req'

In [45]:
combos = combos_6j(trsm_set, n=False, fast_build=True, tag=tag)

  0%|                                                                    | 0/412834 [00:11<?, ?it/s]


ValueError: cannot broadcast records in this type of operation

(https://github.com/scikit-hep/awkward-1.0/blob/1.3.0/src/awkward/_util.py#L947)

In [ ]:
combos7.apply_6j_model(tag)
scores_7 = combos7.scores_combo

In [ ]:
combos7.select_highest_scoring_combos()

In [ ]:
evt_score_mask = combos7.high_score_combo_mask
evt_high_score = combos7.evt_highest_score

In [ ]:
sgnl_evt = combos7.sgnl_evt
sgnl_high_score = combos7.sgnl_high_score

In [ ]:
def get_stats(score_cut = 0):
    # what percentage of events contain signal jets
    percent_signal = np.sum(sgnl_evt)/len(sgnl_evt) * 100
    
    # what percentage of signal events are the highest scoring
    percent_signal_highest_score = np.sum(sgnl_high_score)/np.sum(sgnl_evt) * 100
    
    # score cut removes this percentage
    percent_removal = (1 - np.sum(evt_high_score > score_cut)/len(evt_high_score)) * 100
    
    # of events with highest score above threshold, what percentage of those contain signal combo?
    percent_high_score_are_signal = np.sum(np.logical_and(sgnl_evt, evt_high_score > score_cut))/np.sum(evt_high_score > score_cut) * 100
    
    # of events with highest score above threshold, what percentage of those are signal combos with the highest score in the event?
    percent_high_score_signal_with_highest_score = np.sum(np.logical_and(sgnl_high_score, evt_high_score > score_cut))/np.sum(evt_high_score > score_cut) * 100
    
    print(f"{percent_signal:.0f}% of all events contain signal combos")
    print(f"{percent_signal_highest_score:.0f}% of events containing signal combo assigned highest score to signal combo")
    print(f"{percent_removal:.0f}% of events removed by applying score cut {score_cut}")
    print(f"{percent_high_score_are_signal:.0f}% of events above score cut {score_cut} contain signal combo")
    print(f"{percent_high_score_signal_with_highest_score:.0f}% of events above score cut {score_cut} contain signal combo assigned highest score")

In [ ]:
get_stats(0.8)

In [ ]:
fig, ax = plt.subplots()

n_signal, edges = np.histogram(evt_high_score[sgnl_evt], bins=score_bins)
n_bkgd, edges   = np.histogram(evt_high_score[~sgnl_evt], bins=score_bins)

x = (edges[1:] + edges[:-1])/2

n_signal = n_signal / np.sum(n_signal)
n_bkgd = n_bkgd / np.sum(n_bkgd)

n_signal, edges, _ = hist(ax, x, weights=n_signal, bins=score_bins, label='Events with correct combos')
n_bkgd, edges, _ = hist(ax, x, weights=n_bkgd, bins=score_bins, label='Events with no correct combos')

ax.legend(loc=2)
ax.set_xlabel('Highest Assigned Score Amongst Combinations in Events')
ax.set_ylabel('AU')
ax.set_title('Distribution of Highest Scoring Combination in Events')

fig.savefig('evt_max_score_dist.pdf')

In [ ]:
fig, ax = plt.subplots()

n_signal, edges = np.histogram(evt_high_score[sgnl_evt], bins=score_bins)
n_bkgd, edges   = np.histogram(evt_high_score[sgnl_evt & sgnl_high_score], bins=score_bins)

x = (edges[1:] + edges[:-1])/2

n_signal = n_signal / np.sum(n_signal)
n_bkgd = n_bkgd / np.sum(n_bkgd)

n_signal, edges, _ = hist(ax, x, weights=n_signal, bins=score_bins, label='Events with correct combos')
n_bkgd, edges, _ = hist(ax, x, weights=n_bkgd, bins=score_bins, label='Events with no correct combos')

ax.legend(loc=2)
ax.set_xlabel('Highest Assigned Score Amongst Combinations in Events')
ax.set_ylabel('AU')
ax.set_title('Distribution of Highest Scoring Combination in Events')

In [ ]:
def norm_hist(arr, bins=100):
    n, b = np.histogram(arr, bins=bins)
    x = (b[:-1] + b[1:]) / 2
    
    return n/n.max(), b, x

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6,7))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n_7, b_7, x_7 = norm_hist(scores_7)
c_7, b_7, x_7 = norm_hist(scores_7[combos7.sgnl_mask])
w_7, b_7, x_7 = norm_hist(scores_7[~combos7.sgnl_mask])

hist(ax, x_7, weights=n_7, bins=b_7, label='All 6-jet combos')
hist(ax, x_7, weights=c_7, bins=b_7, label='Correct 6-jet combo')
hist(ax, x_7, weights=w_7, bins=b_7, label='Incorrect 6-jet combo')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Assigned Score')
ax.set_ylabel('AU')

textstr = f'Entries = {len(scores_7)}'
props = dict(boxstyle='round', facecolor='white', alpha=1)
ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
        verticalalignment='top', bbox=props)

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations")

n_7, b_7 = np.histogram(scores_7, bins=100)
c_n_7, b_7 = np.histogram(scores_7[combos7.sgnl_mask], bins=100)
w_n_7, b_7 = np.histogram(scores_7[~combos7.sgnl_mask], bins=100)

x_7 = (b_7[1:] + b_7[:-1]) / 2

hist(ax, x_7, weights=n_7, bins=b_7, label='All 6-jet combos')
hist(ax, x_7, weights=c_n_7, bins=b_7, label='Correct 6-jet combo')
hist(ax, x_7, weights=w_n_7, bins=b_7, label='Incorrect 6-jet combo')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Assigned Score')
ax.set_ylabel('Number of Combinations Per Bin')

textstr = f'Entries = {len(scores_7)}'
props = dict(boxstyle='round', facecolor='white', alpha=1)
ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
        verticalalignment='top', bbox=props)

plt.tight_layout()
fig.savefig('score_dist.pdf')
plt.show()

In [ ]:
pt_7  = combos7.combo_features[:,0:6]
eta_7 = combos7.combo_features[:,6:12]
phi_7 = combos7.combo_features[:,12:18]
m_7   = np.ones(pt_7.shape)*4

In [ ]:
import vector

In [ ]:
p4_7_0 = vector.obj(pt=pt_7[:,0], eta=eta_7[:,0], phi=phi_7[:,0], mass=m_7[:,0])
p4_7_1 = vector.obj(pt=pt_7[:,1], eta=eta_7[:,1], phi=phi_7[:,1], mass=m_7[:,1])
p4_7_2 = vector.obj(pt=pt_7[:,2], eta=eta_7[:,2], phi=phi_7[:,2], mass=m_7[:,2])
p4_7_3 = vector.obj(pt=pt_7[:,3], eta=eta_7[:,3], phi=phi_7[:,3], mass=m_7[:,3])
p4_7_4 = vector.obj(pt=pt_7[:,4], eta=eta_7[:,4], phi=phi_7[:,4], mass=m_7[:,4])
p4_7_5 = vector.obj(pt=pt_7[:,5], eta=eta_7[:,5], phi=phi_7[:,5], mass=m_7[:,5])

In [ ]:
p4_7 = p4_7_0 + p4_7_1 + p4_7_2 + p4_7_3 + p4_7_4 + p4_7_5

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax[0].set_title("Signal 7 Combos")
ax[1].set_title("Background 7 Combos")

n, xedges, yedges, ims = hist2d(ax[0], p4_7.mass[combos7.sgnl_mask], scores_7[combos7.sgnl_mask], xbins=np.linspace(400,900,100))
n, xedges, yedges, imb = hist2d(ax[1], p4_7.mass[~combos7.sgnl_mask], scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,2000,100))

plt.colorbar(ims, ax=ax[0])
plt.colorbar(imb, ax=ax[1])

ax[0].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[1].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[0].set_ylabel('Assigned Score')
ax[1].set_ylabel('Assigned Score')

fig.savefig('score_v_mass.pdf')

plt.show()

In [ ]:
score_bins = np.arange(0,1.01,0.01)

In [ ]:
from numba import jit

@jit(forceobj=True)
def calculate_confusion_matrix(forceobj=True):
    tpr_arr = []
    fpr_arr = []
    prec_arr = []
    recall_arr = []
    for i in score_bins:
        true_pos  = np.sum(scores_7[combos7.sgnl_mask] > i)
        false_pos = np.sum(scores_7[~combos7.sgnl_mask] > i)
        true_neg  = np.sum(scores_7[~combos7.sgnl_mask] <= i)
        false_neg = np.sum(scores_7[combos7.sgnl_mask] <= i)
        
        tpr = true_pos / (true_pos + false_neg) * 100
        fpr = false_pos / (false_pos + true_neg) * 100
        
        tpr_arr.append(tpr)
        fpr_arr.append(fpr)
        
        precision = true_pos / (true_pos + false_pos)
        recall = true_pos / (true_pos + false_neg)
        
        prec_arr.append(precision)
        recall_arr.append(recall)
        
        
    tpr = np.asarray(tpr_arr)
    fpr = np.asarray(fpr_arr)
    precision = np.asarray(prec_arr)
    recall = np.asarray(recall_arr)
    
    f1_score = 2 * precision * recall / (precision + recall)
    
    return tpr, fpr, precision, recall, f1_score
    
tpr, fpr, prec, rec, f1 = calculate_confusion_matrix()

In [ ]:
auc = round(np.sum( (tpr[:-1] + tpr[1:]) / 2 * (fpr[:-1] - fpr[1:]) ) / 100, 3)

F1 is harmonic mean of precision and recall

In [ ]:
fig, ax = plt.subplots()
ax.set_title(r"$^7C_6$ Combinations ROC")

ax.plot(score_bins, prec, label='precision')
ax.plot(score_bins, rec, label='recall')
ax.plot(score_bins, f1, label='F1')

ax.legend()
ax.set_xlabel('Score Threshold')

In [ ]:
fig, ax = plt.subplots()
ax.set_title(r"$^7C_6$ Combinations ROC")

y = np.sqrt(tpr/100*(1-fpr/100))

ax.plot(score_bins, y)
ax.set_ylabel('G-Mean')
ax.set_xlabel('Score Threshold')

In [ ]:
fig, ax = plt.subplots()
ax.set_title(r"$^7C_6$ Combinations ROC")

ax.plot(fpr, tpr)
# ax.scatter(fpr[::10], tpr[::10], s=6, color='orange')
ax.set_ylabel('True Positives %')
ax.set_xlabel('False Positives %')

props = dict(boxstyle='round', facecolor='white', alpha=0.5)
ax.text(0.75, 0.1, f"auc = {auc}%", transform=ax.transAxes, bbox=props)

fig.savefig('roc.pdf')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9,5))

n, b, _ = hist(ax, p4_7.mass[combos7.sgnl_mask], bins=np.linspace(400,900,100), label='Correct Combinations', color='limegreen')
n, b, _ = hist(ax, p4_7.mass[~combos7.sgnl_mask], bins=np.linspace(400,900,100), label='Incorrect Combinations', color='lightcoral')


ax.set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax.set_ylabel('Count')
ax.legend(loc=2)

fig.savefig('inv_mass_after_score_cut.pdf')

plt.show()

In [ ]:
len(evt_score_mask)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9,5))

n, b, _ = hist(ax, p4_7.mass[evt_score_mask][sgnl_evt], bins=np.linspace(400,900,100), label='Correct Highest Score', color='limegreen')
n, b, _ = hist(ax, p4_7.mass[evt_score_mask][~sgnl_evt], bins=np.linspace(400,900,100), label='Incorrect Highest Score', color='lightcoral')


ax.set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax.set_ylabel('Count')
ax.legend(loc=2)

fig.savefig('inv_mass_after_score_cut.pdf')

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9,5))

score_cut_mask = evt_high_score > 0.8
sgnl_score_cut = sgnl_evt & score_cut_mask
bkgd_score_cut = (~sgnl_evt) & score_cut_mask

n, b, _ = hist(ax, p4_7.mass[evt_score_mask][sgnl_score_cut], bins=np.linspace(400,900,100), label='Correct Highest Score > 0.8', color='limegreen')
n, b, _ = hist(ax, p4_7.mass[evt_score_mask][cbkgd_score_cut], bins=np.linspace(400,900,100), label='Incorrect Highest Score > 0.8', color='lightcoral')


ax.set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax.set_ylabel('Count')
ax.legend(loc=2)

fig.savefig('inv_mass_after_score_cut.pdf')

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax[0].set_title("Correct Combos")
ax[1].set_title("Incorrect Combos")

n, b, _ = hist(ax[0], p4_7.mass[combos7.sgnl_mask], bins=np.linspace(400,900,100), label='No cut', color='limegreen')
n, b, _ = hist(ax[0], p4_7.mass[combos7.sgnl_mask][scores_7[combos7.sgnl_mask] > 0.8], bins=np.linspace(400,900,100), label='score > 0.8', color='darkgreen')
n, b, _ = hist(ax[1], p4_7.mass[~combos7.sgnl_mask], bins=np.linspace(400,900,100), label='No cut', color='lightcoral')
n, b, _ = hist(ax[1], p4_7.mass[~combos7.sgnl_mask][scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(400,900,100), label='score > 0.8', color='maroon')
# n, xedges, yedges, imb = hist2d(ax[1], p4_7.mass[~combos7.sgnl_mask], scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,2000,100))


ax[0].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[1].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[0].set_ylabel('Count')
ax[1].set_ylabel('Count')
ax[0].legend()
ax[1].legend()

fig.savefig('inv_mass_postscorecut.pdf')

plt.show()

In [ ]:
evt_score_mask

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6,5), sharex=True, gridspec_kw={'height_ratios':[3,1]})

ns, b, _ = hist(ax[0], p4_7.mass[combos7.sgnl_mask][scores_7[combos7.sgnl_mask] > 0.8], bins=np.linspace(400,900,100), label='sgnl score > 0.8', color='darkgreen')
nb, b, _ = hist(ax[0], p4_7.mass[~combos7.sgnl_mask][scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(400,900,100), label='bkgd score > 0.8', color='maroon')
# n, xedges, yedges, imb = hist2d(ax[1], p4_7.mass[~combos7.sgnl_mask], scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,2000,100))

x = (b[1:] + b[:-1])/2

ax[1].plot(x, ns/nb)
ax[1].plot(x, np.ones_like(x), '--k', alpha=0.2)

ax[1].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[0].set_ylabel('Count')
ax[0].legend()

ax[1].set_ylabel(r'Ratio $S/B_C$')

plt.show()

In [ ]:
# plt.close('all')

In [ ]:
combos7.create_pairs(tag)
# combos7.create_pairs(tag)

In [ ]:
combos7.pair_features.shape

In [ ]:
combos7.apply_2j_model('20210817_4btag_req')

In [ ]:
combos7.scores_pairs[:,0]

In [ ]:
combos7.pair_target

In [ ]:
combos7.select_highest_scoring_pairs()

In [ ]:
# np.savez("../../inputs_2jet_train.npz", inputs=combos7.pair_features, targets=combos7.pair_target)

In [ ]:
# pairs = pairs_2j(combos7)

In [ ]:
len(combos7.all3Higgs_mask[combos7.all3Higgs_mask > 0])/len(combos7.all3Higgs_mask)

In [ ]:
combos7.pair_target[combos7.pair_target > -1]

In [ ]:
from matplotlib import ticker

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,3))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations with Pairing, Normalized", pad=10)

bins = np.linspace(0,1.01,100)

scores_2_noscorecut = combos7.scores_pairs[:,0]


n_2_all, b_2_all, x_2_all = norm_hist(scores_2_noscorecut)
c_2_all, b_2_all, x_2_all = norm_hist(scores_2_noscorecut[combos7.pair_target > -1])
w_2_all, b_2_all, x_2_all = norm_hist(scores_2_noscorecut[combos7.pair_target == -1])

hist(ax, x_2_all, weights=n_2_all, bins=b_2_all, label='All pairs')
hist(ax, x_2_all, weights=c_2_all, bins=b_2_all, label='Correct pairs')
hist(ax, x_2_all, weights=w_2_all, bins=b_2_all, label='Incorrect pairs')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Assigned Score')
ax.set_ylabel('AU')

textstr = f'Entries = {len(scores_2_noscorecut)}'
props = dict(boxstyle='round', facecolor='white', alpha=1)
ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
        verticalalignment='top', bbox=props)

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations with Pairing", pad=10)

formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True) 
formatter.set_powerlimits((-1,1)) 
ax.yaxis.set_major_formatter(formatter) 

n_2_all, b_2_all = np.histogram(scores_2_noscorecut, bins=100)
c_2_all, b_2_all = np.histogram(scores_2_noscorecut[combos7.pair_target > -1], bins=100)
w_2_all, b_2_all = np.histogram(scores_2_noscorecut[combos7.pair_target == -1], bins=100)

x_2_all = (b_2_all[1:] + b_2_all[:-1]) / 2

# hist(ax, x_2_all, weights=n_2_all, bins=b_2_all, label='All 6-jet combos')
hist(ax, x_2_all, weights=c_2_all, bins=b_2_all, label='Correct pairs')
hist(ax, x_2_all, weights=w_2_all, bins=b_2_all, label='Incorrect pairs')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Assigned Score')
ax.set_ylabel('Number of Combinations Per Bin')

textstr = f'Entries = {len(scores_2_noscorecut)}'
props = dict(boxstyle='round', facecolor='white', alpha=1)
ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
        verticalalignment='top', bbox=props)

ax.text(0.3, 0.25, f'Ratio correct/total above 0.8 = {100*sum(c_2_all[x_2_all > 0.8])/sum(n_2_all[x_2_all > 0.8]):.0f}%', transform=ax.transAxes)
# ax.ticklabel_format(useMathText=True, useOffset=True)

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,3))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations with Pairing, Normalized")

bins = np.linspace(0,1.01,100)

scores_2 = combos7.scores_pairs[:,0]


n_7, b_7, x_7 = norm_hist(scores_2)
c_n_7, b_7, x_7 = norm_hist(scores_2[combos7.pair_target > -1])
w_n_7, b_7, x_7 = norm_hist(scores_2[combos7.pair_target == -1])

# hist(ax, x_7, weights=n_7, bins=b_7, label='All pairs')
hist(ax, x_7, weights=c_n_7, bins=b_7, label='Correct pairs')
hist(ax, x_7, weights=w_n_7, bins=b_7, label='Incorrect pairs')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Assigned Score')
ax.set_ylabel('AU')

textstr = f'Entries = {len(scores_2)}'
props = dict(boxstyle='round', facecolor='white', alpha=1)
ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
        verticalalignment='top', bbox=props)

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations with Pairing")

n_7, b_7 = np.histogram(scores_2, bins=100)
c_n_7, b_7 = np.histogram(scores_2[combos7.pair_target > -1], bins=100)
w_n_7, b_7 = np.histogram(scores_2[combos7.pair_target == -1], bins=100)

x_7 = (b_7[1:] + b_7[:-1]) / 2

# hist(ax, x_7, weights=n_7, bins=b_7, label='All 6-jet combos')
hist(ax, x_7, weights=c_n_7, bins=b_7, label='Correct pairs')
hist(ax, x_7, weights=w_n_7, bins=b_7, label='Incorrect pairs')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Assigned Score')
ax.set_ylabel('Number of Combinations Per Bin')

textstr = f'Entries = {len(scores_2)}'
props = dict(boxstyle='round', facecolor='white', alpha=1)
ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
        verticalalignment='top', bbox=props)

ax.text(0.3, 50000, f'Ratio correct/total above 0.8 = {100*sum(c_n_7[x_7 > 0.8])/sum(n_7[x_7 > 0.8]):.0f}%')

plt.tight_layout()
plt.show()